## Created by Anupal Baruah, Post Doc, SDML , UA

In [ ]:
!pip install boto3

In [ ]:
!pip install awscli

In [ ]:
import os

# Set your AWS credentials and region

os.environ['AWS_ACCESS_KEY_ID'] = '--------'
os.environ['AWS_SECRET_ACCESS_KEY'] = '--------'
os.environ['AWS_DEFAULT_REGION'] = '-------'

# Verify that the environment variables are set correctly

print("AWS Access Key:", os.environ.get('AWS_ACCESS_KEY_ID'))
print("AWS Secret Access Key:", os.environ.get('AWS_SECRET_ACCESS_KEY'))
print("AWS Default Region:", os.environ.get('AWS_DEFAULT_REGION'))

In [ ]:
## Check the content of AWS S3 bucket

!aws s3 ls s3://noaa-nws-owp-fim/hand_fim/fim_4_4_0_0/ --request-payer

## Download the sample_inputs data from the AWS

In [ ]:
!aws s3 sync s3://noaa-nws-owp-fim/hand_fim/sample_inputs /Users/abaruah/Documents/HAND_Mac/sample_inputs --request-payer

# Run the AWS CLI command to download your AOI

In [ ]:
!aws s3 sync s3://noaa-nws-owp-fim/hand_fim/fim_4_4_0_0/12090301 /Users/abaruah/Documents/HAND_Mac/12090301 --request-payer

In [ ]:
## CHeck the git version
!git --version

# Clone from the git hub from this branch "dev-whitebox-2.3.4" to ignore the whitebox error. You need to comment line 103 to 118 in the dockerfile if not commented

In [ ]:
!git clone https://github.com/NOAA-OWP/inundation-mapping.git /Users/abaruah/Documents/HAND_Mac

## Build the docker image from the Dockerfile 

In [ ]:
import os

image_name = "sample_inputs"
tag = "latest"
path_to_repository = "/Users/abaruah/Documents/HAND_Mac/inundation-mapping/"

os.chdir(path_to_repository)

docker_build_command = f"docker build -f Dockerfile -t {image_name}:{tag} ."

!{docker_build_command}

## Preaparing the feature_id file from the hydrotable

In [ ]:
import pandas as pd

hydrotable_ = pd.read_csv(r'/Users/abaruah/Documents/HAND_Mac/12090301/hydrotable.csv')

unique_feature_ids = hydrotable_['feature_id'].drop_duplicates()


unique_feature_ids_df = pd.DataFrame(unique_feature_ids, columns=['feature_id'])


print(unique_feature_ids_df.head())


unique_feature_ids_df.to_csv(r'/Users/abaruah/Documents/HAND_Mac/feature_id_12090301.csv', index=False)

In [ ]:
import multiprocessing
import os
import fsspec
import pandas as pd
import xarray as xr
from tqdm import tqdm
 

In [ ]:
out_dir=r'/Users/abaruah/Documents/HAND_Mac'
file_name='feature_id_12090301.csv'
file_path=os.path.join(out_dir,file_name)
fid_12090301=pd.read_csv(file_path)
#print(fid_12090301)

In [ ]:

pip install xarray zarr fsspec s3fs


In [ ]:
fs = fsspec.filesystem('s3', anon=True)
_file = fs.glob('noaa-nwm-retrospective-2-1-zarr-pds/chrtout.zarr')

ds = xr.open_dataset(fs.get_mapper(_file[0]), engine='zarr', backend_kwargs={'consolidated': True})
#print(ds)



# # October 2017
s_date = '2016-10-09T14:00'
e_date = '2016-10-09T16:00'

ds2 = ds.sel(time=slice(s_date, e_date))

directory_name='02050301_OCt_2016flood'

discharge_dir = os.path.join(out_dir + '/' + '02050301_OCt_2016flood/')

os.makedirs(discharge_dir, exist_ok=True)

print(f"Directory created at: {discharge_dir}")


In [ ]:
for index, row in fid_12090301.iterrows():
    feature_id = row['feature_id']
    try:
        Forecast_point = ds2['streamflow'].sel(feature_id=int(feature_id))
        #print(Forecast_point)

        Forecast_point = Forecast_point.to_pandas().to_frame()
        Forecast_point.columns = [feature_id]
        #print(Forecast_point)

        Forecast_point.to_csv(os.path.join(discharge_dir, str(feature_id) + '.csv'))

    except:
        print('\nError with station row : ', index, ' | Feature ID : ', feature_id)
        pass

In [ ]:
import os
import pandas as pd


folder_path = r'/Users/abaruah/Documents/HAND_Mac/12090301_OCt_2016flood'


concatenated_data = pd.DataFrame()


for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
         file_path = os.path.join(folder_path, filename)
        

         df = pd.read_csv(file_path, encoding='latin-1')
        
#         # Extract the discharge column and use the filename as the header
         header = os.path.splitext(filename)[0]  # Use filename without extension as header
         df = df.rename(columns={'discharge': header})
        
#         # Add the data to the concatenated DataFrame
         concatenated_data = pd.concat([concatenated_data, df], axis=1)


concatenated_data = concatenated_data.loc[:,~concatenated_data.columns.duplicated()]



output_file_path = os.path.join(folder_path, '12090301_InputHANDFIM.csv')
concatenated_data.to_csv(output_file_path, index=False)


In [ ]:

import pandas as pd

# Load the second CSV file (xyz.csv)
xyz_df = pd.read_csv(r'/Users/abaruah/Documents/HAND_Mac/12090301_OCt_2016flood/12090301_InputHANDFIM.csv')


transposed_row = xyz_df.iloc[0].transpose()


transposed_row_df = transposed_row.to_frame().reset_index()


transposed_row_df.columns = ['feature_id', 'discharge']

transposed_row_df = transposed_row_df.drop(index=0).reset_index(drop=True)

print(transposed_row_df)
transposed_row_df.to_csv(r'/Users/abaruah/Documents/HAND_Mac/12090301_1stthhour.csv')


# Create a blank folder 'Data' and within the 'Data' folder create sub-folder 'Inputs'. Copy and paste the contents from the sample inputs to 'Inputs'


# Keep the 12090301_1sthour.csv in the rating_curve folder within the Inputs

In [ ]:

# Running the container in detach mode

import subprocess
import time

container_name = "new_run"
image_name = "sample_inputs"
tag = "latest"
volumes = [
    "/Users/abaruah/Documents/HAND_Mac/inundation-mapping/:/foss_fim",
    "/Users/abaruah/Documents/HAND_Mac/output/:/outputs",
    "/Users/abaruah/Documents/HAND_Mac/Outputs_temp/:/fim_temp",
    "/Users/abaruah/Documents/HAND_Mac/Data/:/data"
]

volume_args = " ".join([f"-v {volume}" for volume in volumes])

# Docker run command in detached mode (-d) with a long-running process
docker_run_command = f"docker run -d --name {container_name} {volume_args} {image_name}:{tag} tail -f /dev/null"

# Print the command 
print("Running command:", docker_run_command)

# Execute the command to run the container
result = subprocess.run(docker_run_command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Print the output and error (if any)
print(result.stdout.decode())
if result.stderr:
    print(result.stderr.decode())

# Check if the command was successful
if result.returncode == 0:
    print("Docker container started successfully.")
    # Wait for 15 seconds (adjust as needed)
    time.sleep(15)
    # Stop the execution of the cell
    raise SystemExit("Stopped the cell execution. Container is still running.")
else:
    print("Failed to start the Docker container.")
    print(result.stderr.decode())

# To run the same command from terminal if there is an issue while running from the notebook

In [ ]:
docker run --rm -it --name new_run -v /Users/abaruah/Documents/HAND_Mac/inundation-mapping/:/foss_fim -v /Users/abaruah/Documents/HAND_Mac/output/:/outputs -v /Users/abaruah/Documents/HAND_Mac/Outputs_temp/:/fim_temp -v /Users/abaruah/Documents/HAND_Mac/Data/:/data sample_inputs:latest

In [ ]:
# Within the output folder we need to put the downloaded HUC-8 and one csv file fim_inputs.csv. Rename the branch_id.csv file withiin the downloaded HUC-8 (12090301) folder to fim_inputs.csv 

In [ ]:
# Docker run

command_to_run_inside_container = "python foss_fim/tools/inundate_mosaic_wrapper.py -y /outputs -u 12090301 -f /data/Inputs/rating_curve/bankfull_flows/12090301_1stthhour.csv -i /outputs/flood_12090301/inundation_1sthour_trial_.tif"


docker_exec_command = f"docker exec -it {container_name} {command_to_run_inside_container}"

!{docker_exec_command}

# To run the command from terminal

python foss_fim/tools/inundate_mosaic_wrapper.py -y /outputs -u 12090301 -f /data/Inputs/rating_curve/bankfull_flows/12090301_1stthhour.csv -i /outputs/flood_12090301/inundation_1sthour_trial_.tif
